In [ ]:
import pandas as pd
#from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit

## Model

In [ ]:
# %load "features/clicksPerAd.py"
def clicksPerAd(clicks):
    return pd.DataFrame({'clicks': clicks.groupby(['ad_id'])['clicked'].sum(),
                         'shows': clicks.groupby(['ad_id'])['clicked'].size(),
                         'clicksPerShows': clicks.groupby(['ad_id'])['clicked'].mean()
                        }).reset_index()

In [ ]:
def join(series):
    #return ' '.join( str(list(series)).strip('[]').split(', ') )
    return ' '.join( map(str, series) )

In [ ]:
def model_prediction(by, clicksPerAd_train, test):
    prediction = pd.merge(test, clicksPerAd_train, how='left', on='ad_id').fillna(clicksPerAd_train[by].mean())
    prediction = prediction.sort_values(by=['display_id', by], ascending=False)
    return pd.DataFrame({ 'ad_id': prediction.groupby('display_id')['ad_id'].apply(join) }).reset_index()

In [ ]:
%run 'metrics.ipynb'
def model_evaluation(actual, prediction):
    return mapk(actual['ad_id'], prediction['ad_id'], k=12)

## Validation

In [ ]:
data = pd.read_csv("../download/clicks_train.csv")

In [ ]:
displays = pd.read_csv("../download/events.csv", usecols=['display_id'])
displays.count()

In [ ]:
#kf = KFold(n_splits=5)
#train, test = kf.split(displays).next()
ss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=0)
train_index, test_index = ss.split(displays).next()

print len(train_index), len(test_index)

In [ ]:
clicksPerAd_train = clicksPerAd( pd.merge(data, displays.iloc[train_index], on=['display_id']) )
clicks_test_sample = pd.merge(data, displays.iloc[test_index], on=['display_id'])

validation = clicks_test_sample[clicks_test_sample['clicked']==1].astype({'ad_id':str})

In [ ]:
by='clicksPerShows'
print by, ':', model_evaluation(validation, model_prediction(by, clicksPerAd_train, clicks_test_sample))

## Submission

In [ ]:
clicksPerAd_train = pd.read_csv("../generated/clicks_shows_PerAd.csv", usecols=['ad_id', 'clicksPerShows'])
clicks_test = pd.read_csv("../download/clicks_test.csv")
prediction = model_prediction('clicksPerShows', clicksPerAd_train, clicks_test)
prediction.head(20)

In [ ]:
prediction.to_csv("../submissions/solution_2.csv", index=False)